# TSA Chapter 3: Python Lab: Unit Root Testing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch3/TSA_ch3_python_unit_root/TSA_ch3_python_unit_root.ipynb)

This notebook demonstrates:
- ADF and KPSS unit root tests on stationary AR(1) and random walk
- Complementary test interpretation.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 300

eps = np.random.normal(0, 1, n)
stationary = np.zeros(n)
for t in range(1, n):
    stationary[t] = 0.8 * stationary[t-1] + eps[t]
random_walk = np.cumsum(eps)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, s, title in zip(axes, [stationary, random_walk], ['AR(1) phi=0.8', 'Random Walk']):
    ax.plot(s, color='#1A3A6E', linewidth=0.8)
    ax.set_title(title, fontweight='bold')

    adf = adfuller(s, regression='c')
    try:
        kpss_stat, kpss_p, _, _ = kpss(s, regression='c', nlags='auto')
    except Exception:
        kpss_p = float('nan')

    ax.text(0.02, 0.98,
            f'ADF: stat={adf[0]:.2f}, p={adf[1]:.4f}\nKPSS: p={kpss_p:.4f}',
            transform=ax.transAxes, fontsize=9, va='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
save_chart(fig, 'sem3_adf_kpss')
plt.show()

print("ADF: H0 = unit root. Reject if p < 0.05 -> stationary")
print("KPSS: H0 = stationary. Reject if p < 0.05 -> unit root")
print("Use both together for robust conclusions.")